# Подготовка к циклу

In [28]:
from gzip import open # NB: overrides standard open()
import pickle as pkl
import tensorly as tl
from tensorly.base import tensor_to_vec,  partial_tensor_to_vec
import pandas as pd
import warnings 
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import r2_score
from tensorly.regression.kruskal_regression import KruskalRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

In [29]:
#формируем логарифмическую шкалу для tol
log_space=np.logspace(-6,6,3,endpoint=True)

In [30]:
#формируем логарифмическую шкалу для reg_W
log_space2=np.logspace(-6,3,4,endpoint=True)

In [31]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cd  %clear  %cls  %colors  %conda  %config  %connect_info  %copy  %ddir  %debug  %dhist  %dirs  %doctest_mode  %echo  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %macro  %magic  %matplotlib  %mkdir  %more  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %ren  %rep  %rerun  %reset  %reset_selective  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%cmd  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python 

In [32]:
warnings.simplefilter('ignore') # отключим предупреждения Anaconda

# Непосредственно сама обработка данных

## Вынесу класс оборачивающий регрессию в обдельную клетку

In [33]:
class WrappedKruskalRegressor(BaseEstimator):
	def __init__(self, weight_rank = 1, tol = 1e-6, reg_W = 1, n_iter_max = 100):
		self.weight_rank = weight_rank
		self.tol = tol
		self.reg_W = reg_W
		self.n_iter_max = n_iter_max
	
	def fit(self, X, y = None):
		self._model = KruskalRegressor(self.weight_rank, self.tol, self.reg_W, self.n_iter_max)
		self._model.fit(X, y)
		if self._model.n_iterations_ == self.n_iter_max:
			raise Exception('{} did not converge'.format(self))
		return self
	
	def predict(self, X):
		return self._model.predict(X)
	
	def score(self, X, y):
		return r2_score(y, self.predict(X))

## Начинаем применять класс

In [35]:
%%time
Xdata_numpy = pkl.load(open('C:/Users/admin/Desktop/88/X.pkl.gz', 'rb'))
Ydata = pkl.load(open('C:/Users/admin/Desktop/88/y.pkl.gz', 'rb'))

print(type(Xdata_numpy["X"]))

#делим набор на обучающий и тестовый
X_train, X_test, y_train, y_test = train_test_split(
     Xdata_numpy["X"], Ydata, test_size=0.2857 
)

print( y_train.shape)

#начинаем применять
regressioModel=WrappedKruskalRegressor()
parametrsNames={'n_iter_max': [300000],
                'reg_W': log_space2,
                'tol': log_space,
                'weight_rank': range(1,6)
               }

gridCought=GridSearchCV(regressioModel, parametrsNames, cv=5)
gridCought.fit(X_train,y_train.iloc[:,0])
predictors=gridCought.predict(X_test)

print(gridCought.best_params_)

<class 'numpy.ndarray'>
(25, 3)

Converged in 84047 iterations

Converged in 415 iterations

Converged in 4001 iterations

Converged in 3725 iterations

Converged in 3056 iterations

Converged in 17 iterations

Converged in 27 iterations

Converged in 10 iterations

Converged in 14 iterations

Converged in 36 iterations

Converged in 22 iterations

Converged in 244 iterations

Converged in 22 iterations

Converged in 19 iterations

Converged in 14 iterations

Converged in 10 iterations

Converged in 23 iterations

Converged in 108 iterations

Converged in 229 iterations

Converged in 304 iterations

Converged in 49 iterations

Converged in 28 iterations

Converged in 18 iterations

Converged in 47 iterations

Converged in 43 iterations

Converged in 2 iterations

Converged in 2 iterations

Converged in 2 iterations

Converged in 2 iterations

Converged in 2 iterations

Converged in 2 iterations

Converged in 2 iterations

Converged in 2 iterations

Converged in 2 iterations

Converged 


Converged in 2 iterations

Converged in 2 iterations

Converged in 2 iterations

Converged in 2 iterations

Converged in 2 iterations
{'n_iter_max': 300000, 'reg_W': 1.0, 'tol': 1.0, 'weight_rank': 2}
Wall time: 1h 15min 34s


In [ ]:
gridCought.best_estimator_

In [ ]:
gridCought.score(X_test, y_test.iloc[:,0])

In [ ]:
plt.plot(y_test.iloc[:,0],y_test.iloc[:,0],color="blue")
plt.plot(y_test.iloc[:,0],predictors,".",color="red")
plt.xlabel("Истинные значения",fontsize=12)
plt.ylabel("Предсказанные значения", fontsize=12)
plt.grid()
plt.show();

In [ ]:
gridCought.scorer_

In [ ]:
gridCought.score

In [ ]:
 gridCought.cv_results_

In [ ]:
gridCought.cv_results_['mean_test_score']

In [ ]:
cvres  = gridCought.cv_results_

In [ ]:
gridCought.cv_results_['mean_test_score'][6]

In [ ]:
min(gridCought.cv_results_['mean_test_score'])

In [ ]:
for mean_score,  params  in zip(cvres [ "mean_test_score" ], 
                                cvres [ "params" ]
                                ): 
    print (np.sqrt(mean_score**2) , params) 

In [ ]:
score=[]
score_l=[]
parametr_reg_w=[]
my_error=[]

In [ ]:
def f(x,y):
    my_error=[]
    for i in range(0,len(y)):
        my_error+=[(x[i]-y[i])**2]
    return my_error
my_error=f(np.array(y_test.iloc[:,0]),predictors)
print(my_error)

In [ ]:
for mean_score,  params  in zip (cvres [ "mean_test_score"],cvres [ "params" ] )  : 
    score+=[np.array(mean_score)**2]
    score_l+=[np.array((mean_score))]
    parametr_reg_w+=[params["reg_W"]]
print (score,"   ",parametr_reg_w) 


In [ ]:
plt.plot(parametr_reg_w,score_l,".",color="red")
#plt.plot(parametr_reg_w,score_l,".",color="blue")
plt.grid()
plt.ylabel("Ошибка",fontsize=15)
plt.xlabel("Параметр",fontsize=15)
plt.title("График зависимости квадрата ошибки от reg_W",fontsize=18)
plt.show();

In [ ]:
?file_obj.write(plt)